In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
pd.set_option("display.max_columns", None)
import json
import folium
import warnings
warnings.filterwarnings("ignore")
import requests
import os
import plotly.express as px
import tkinter as tk
from tkinter import filedialog
from selectolax.parser import HTMLParser
from folium.features import CustomIcon
from folium.plugins import MarkerCluster
import branca
import altair as alt

### 讀取BS的FMC並使用tomtom的api來使用地址返回經緯度

In [7]:
# bs_shops = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\行銷推廣\掛牌店\20230712_FMC全部名單.csv")
bs_shops = pd.read_csv("/Users/kai/Library/CloudStorage/OneDrive-Bridgestone/行銷推廣/掛牌店/20230712_FMC全部名單.csv")

In [8]:
def get_geometry(address):
        url = 'https://api.tomtom.com/search/2/geocode/{}.json?&key=k0ZVFafBkRAmK0rAWeA2yC2CZTkRCxhc&countrySet=TWN&language=zh-TW&limit=1'.format(
                address)
        res = requests.get(url)
        return tuple(reversed(res.json()["results"][0]["position"].values()))

In [9]:
bs_shops["geometry"] = bs_shops["地址"].map(get_geometry)

In [10]:
bs_shops["geometry"] = bs_shops["geometry"].map(lambda x: Point(x))

In [11]:
bs_shops["店別"] = bs_shops["大胎契約"].map(lambda x: "BFP" if x == 3 else "TTC")

In [13]:
bs_shops.to_csv("/Users/kai/Library/CloudStorage/OneDrive-Bridgestone/行銷推廣/掛牌店/20230712_FMC全部名單.csv", index=False, encoding="utf-8-sig")

### 2021-2023YTD銷售

In [15]:
# sales = pd.read_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet")
sales = pd.read_parquet("/Users/kai/Library/CloudStorage/OneDrive-Bridgestone/數據/銷售數據/長期銷售記錄/2017-202208.parquet")

In [16]:
cus_list = ['C1B10329', 'C1C10413', 'C2A10084', 'C2A10239', 'C2A10526',
       'C2A10568', 'C2B10223', 'C2B10766', 'C2B10875', 'E1E10113',
       'E1E10333', 'E1E10636', 'E1E10064', 'E1E10120', 'E1E10138',
       'E1E10150', 'E1E10276', 'F1F10346', 'F1F10391', 'F1F10426',
       'F1F10435', 'F1F10470', 'F1F10139', 'F1F10451', 'H0G1A022',
       'H0G1B024', 'H0G1D045', 'H0G1E056', 'H0H1A006', 'H0H1A019',
       'H0H1A034', 'H0H1A036', 'H0H1B110', 'H0H1C019', 'I1I1A012',
       'I1I1B023', 'I1I1B113', 'I1I1B134', 'J1J10232', 'J1J10279',
       'J1J10420', 'J1J10690', 'J1J10763', 'J1J10771', 'K1K10170',
       'K1K10989', 'K1K11013', 'L1L1A006', 'L1L1B040', 'L1L1B051',
       'L1L1B074', 'L1L1B092', 'L1L1C111', 'L1L1D027', 'L1L1D051',
       'L1L1G033', 'L1L1H024', 'L1L1H025', 'L1L1H032', 'N1N1B105']

In [17]:
plot_df = sales[(sales["實際出貨日"] >= "2021-01-01") & (sales["買方"].isin(cus_list))].groupby([pd.Grouper(key="實際出貨日", freq="YS"), "買方"])[["銷售數量"]].sum().reset_index()

In [26]:
plot_df["year"] = plot_df["實際出貨日"].dt.year

In [30]:
plot_df["year"] = plot_df["year"].astype(str)

In [49]:
def plotting(id):
    p = plot_df[plot_df["買方"] == id]
    fig = px.bar(p, x="實際出貨日", y="銷售數量")
    base_path = r"C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots"
    file_name = "\\" + id + ".html"
    full_path = base_path + file_name
    fig.update_layout(margin=dict(t=30,l=10,b=10,r=10))
    fig.to_html(full_path)
    return full_path

In [50]:
bs_shops["plot"] = bs_shops["客戶編號"].map(plotting)

In [51]:
bs_shops

,客戶編號,客戶名稱,販社通路,大胎契約,業務名稱,地址,geometry,店別,plot
0,C1B10329,五七輪胎行,H,3,林俊良,桃園市龜山區文化三路416號,POINT (121.36703 25.05366),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\C1B10329.html
1,C1C10413,勇和實業有限公司,H,3,林俊良,新北市泰山區新五路一段20號,POINT (121.44056 25.0616),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\C1C10413.html
2,C2A10084,麒駪輪胎行,H,3,林俊良,新北市新店區安業街155-2號,POINT (121.52157 24.9738),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\C2A10084.html
3,C2A10239,協助輪胎有限公司,H,3,林俊良,新北市三重區環河北路二段330號,POINT (121.49799 25.08364),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\C2A10239.html
4,C2A10526,寬圍輪胎行,H,3,林俊良,新北市汐止區大同路三段453號,POINT (121.67869 25.07145),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\C2A10526.html
...,...,...,...,...,...,...,...,...,...
55,L1L1G033,安速輪胎有限公司,H,3,張正平,高雄市仁武區鳳仁路72-6號,POINT (120.35524 22.6828),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\L1L1G033.html
56,L1L1H024,信興輪胎行,H,3,張正平,高雄市路竹區中山路1303號,POINT (120.26198 22.85156),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\L1L1H024.html
57,L1L1H025,龍城輪胎行,H,3,張正平,高雄市路竹區環球路86之7號,POINT (120.27179 22.87838),BFP,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\L1L1H025.html
58,L1L1H032,越大輪胎有限公司,H,2,張正平,高雄市岡山區介壽東路91-1號,POINT (120.31249 22.78538),TTC,C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\plots\L1L1H032.html


In [37]:
geo_bs_shops = gpd.GeoDataFrame(bs_shops, crs=4326, geometry=bs_shops["geometry"])

In [38]:
geojson_bs = json.loads(geo_bs_shops.to_json())

### 讀取米其林輪胎店經緯度

In [39]:
url = "https://www.michelin.com.tw/truck/api/getdealer.php"
data = {"category": "tryeplus"}
res = requests.post(url, data=data)
mi_shops = pd.DataFrame(res.json()["data"])
mi_shops["lat"] = mi_shops["lat"].map(lambda x: x.split("+")[0] if "+" in x else x)
mi_shops["geometry"] = mi_shops.apply(lambda x: Point(x["lng"], x["lat"]), axis=1)
mi_shops = mi_shops[["name", "county", "district", "address", "geometry"]]

In [40]:
# pd.DataFrame to gpd.GeoDataFrame
geo_mi_shops = gpd.GeoDataFrame(mi_shops, crs=4326, geometry=mi_shops["geometry"])

In [41]:
geojson_mi = json.loads(geo_mi_shops.to_json())

### 讀取瑪吉斯輪胎店經緯度

In [42]:
def get_total_pages():
    res = requests.get("https://www.cst.com.tw/dealers.php?city=0&area=0&tire=32")
    content = HTMLParser(res.text)
    page = int(content.css_first("ul.pagination.pagination-lg > li:nth-last-child(2)").text())
    page += 1
    return page

In [43]:
def get_ma_shops():
    datas = []
    pages = get_total_pages()
    for page in range(1, pages):
        url = "https://www.cst.com.tw/dealers.php?city=0&area=0&tire=32&page={}".format(page)
        res = requests.get(url)
        content = HTMLParser(res.text)
        for c in content.css("div.row > div.col-md-6 > div.row > div.col-md-6:nth-child(2n)"):
            name = c.css_first("h4").text()
            tel = c.css_first("p.txt_org").text()
            address = c.css_first("p:nth-last-child(2)").text()
            data = [name, tel, address]
            datas.append(data)
    df = pd.DataFrame(datas, columns=["name", "tel", "address"])
    return df
    

In [44]:
ma_shops = get_ma_shops()

In [45]:
ma_shops["geometry"] = ma_shops["address"].map(get_geometry)

In [46]:
ma_shops["geometry"] = ma_shops["geometry"].map(lambda x: Point(x))

In [47]:
geo_ma_shops = gpd.GeoDataFrame(ma_shops, crs=4326, geometry=ma_shops["geometry"])

In [48]:
geojson_ma = json.loads(geo_ma_shops.to_json())

### 讀取台灣縣市界圖的shp

In [54]:
## 縣市界圖
# city_map = gpd.read_file(r"C:\Users\kc.hsu\Downloads\COUNTY_MOI_1090820.shp", encoding="utf-8")
city_map = gpd.read_file("https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=72874C55-884D-4CEA-B7D6-F60B0BE85AB0", encoding="utf-8")
# city_map = city_map.to_crs({"init": "epsg:4326"})

In [26]:
## 鄉鎮市區界圖
def get_geo_data():
    cc = gpd.read_file(r"C:\Users\kc.hsu\Python workspace\sales_related\Marketing相關\geopandas\geo_datasets\TOWN_MOI_1120317.shp", encoding="utf-8") # 鄉鎮市區界圖(TWD97經緯度)
    cc = cc.to_crs({"init": "epsg:4326"})
    # cc_area = cc.copy()
    # cc_area = cc_area.to_crs(epsg=3826)
    # cc_area["area"] = round(cc_area["geometry"].area / 10 ** 6, 2)
    # cc["area"] = cc_area["area"]
    return cc

In [55]:
city_map

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry
0,Z,09007,連江縣,Lienchiang County,"MULTIPOLYGON (((119.96445 25.94552, 119.96427 ..."
1,G,10002,宜蘭縣,Yilan County,"MULTIPOLYGON (((121.95971 24.84494, 121.96064 ..."
2,N,10007,彰化縣,Changhua County,"POLYGON ((120.45655 24.20719, 120.48589 24.197..."
3,M,10008,南投縣,Nantou County,"POLYGON ((121.27086 24.23661, 121.27108 24.236..."
4,P,10009,雲林縣,Yunlin County,"MULTIPOLYGON (((120.08108 23.52412, 120.08161 ..."
5,C,10017,基隆市,Keelung City,"MULTIPOLYGON (((121.71015 25.17604, 121.71015 ..."
6,A,63000,臺北市,Taipei City,"POLYGON ((121.57098 25.19717, 121.57097 25.197..."
7,F,65000,新北市,New Taipei City,"POLYGON ((121.53752 25.30003, 121.53768 25.300..."
8,B,66000,臺中市,Taichung City,"POLYGON ((121.32834 24.43304, 121.32845 24.432..."
9,D,67000,臺南市,Tainan City,"POLYGON ((120.44025 23.41308, 120.44040 23.413..."


In [28]:
# city_map = get_geo_data()

## 取得鄉鎮市區戶數、人口

In [50]:
def get_population(geo_data):
    people = pd.read_csv("https://data.moi.gov.tw/MoiOD/System/DownloadFile.aspx?DATA=CA18EE06-4A50-4861-9D97-7853353D7108", header=1) # 鄉鎮市區戶數、人口
    people = people.dropna()
    people = people[(~people["人口密度"].str.contains("…"))].reset_index(drop=True)
    people["城市"] = people["區域別"].map(lambda x: x[:3])
    people = people.astype({"年底人口數": int, "人口密度": int})
    people = people.groupby('城市')[['年底人口數', '土地面積', '人口密度']].sum().reset_index()
    result= pd.merge(people, geo_data, how="left", left_on=["城市"], right_on=["COUNTYNAME"])
    result["土地面積"] = result["土地面積"].round(1)
    result = result[['城市', '年底人口數', '土地面積', '人口密度']]
    return result

In [56]:
population = get_population(city_map)

In [57]:
population

,城市,年底人口數,土地面積,人口密度
0,南投縣,479595,4106.4,4220
1,嘉義市,262924,60.0,8765
2,嘉義縣,488158,1903.6,6588
3,基隆市,361526,132.8,31035
4,宜蘭縣,449062,2143.6,13231
5,屏東縣,798703,2775.6,17993
6,彰化縣,1245239,1074.4,32758
7,新北市,3995551,2052.6,191976
8,新竹市,452473,104.2,17709
9,新竹縣,580503,1427.5,10820


In [58]:
city_map = pd.merge(city_map, population, how="left", left_on="COUNTYNAME", right_on="城市")

In [59]:
## 建立基本資料底圖
cc_geojson = json.loads(city_map.to_json())

In [70]:
geojson_bs["features"][0]["properties"]

{'客戶編號': 'C1B10329',
 '客戶名稱': '五七輪胎行',
 '販社通路': 'H',
 '大胎契約': 3,
 '業務名稱': '林俊良',
 '地址': '桃園市龜山區文化三路416號',
 '店別': 'BFP'}

In [85]:
def alt_plot(id, shop_name):
    p = plot_df[plot_df["買方"] == id]
    chart = alt.Chart(p, title=shop_name).mark_bar().encode(
        x="銷售數量:Q",
        y="year:N",
        tooltip=["銷售數量"]
    )
    return chart.to_dict()
    

In [86]:
ccToolTip = folium.GeoJsonTooltip(["COUNTYNAME", "年底人口數", "土地面積", "人口密度"], ["縣市", "人口數", "面積", "人口密度"])
map = folium.Map([23, 120], zoom_start=8)
folium.GeoJson(cc_geojson, name="basic_info.", tooltip=ccToolTip).add_to(map)
feature_group_bs = folium.FeatureGroup("BS_shops")
for bs in geojson_bs["features"]:
    chart = alt_plot(bs["properties"]["客戶編號"], bs["properties"]["客戶名稱"])
    iframe = folium.IFrame("Shop:" + str(bs["properties"]["客戶名稱"]) + "<br>" + "Type:" + str(bs["properties"]["店別"]) + "<br>" + "Address:" + str(bs["properties"]["地址"]))
    popup = folium.Popup(iframe, max_width=450).add_child(folium.VegaLite(chart))
    folium.Marker(location=list(reversed(bs["geometry"]["coordinates"])), 
                    icon=folium.Icon(color="red"), 
                    popup=popup).add_to(feature_group_bs)
    
feature_group_mi = folium.FeatureGroup("MI_shops")
for mi in geojson_mi["features"]:
    iframe = folium.IFrame("Shop:" + str(mi["properties"]["name"]) + "<br>" + "Type:" + "TyrePlus" + "<br>" + "Address:" + str(mi["properties"]["address"]))
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    folium.Marker(location=list(reversed(mi["geometry"]["coordinates"])), 
                    icon=folium.Icon(color="blue"), 
                    popup=popup).add_to(feature_group_mi)
feature_group_ma = folium.FeatureGroup("MA_shops")
for ma in geojson_ma["features"]:
    iframe = folium.IFrame("Shop:" + str(ma["properties"]["name"]) + "<br>" + "Type:" + "" + "<br>" + "Address:" + str(ma["properties"]["address"]))
    popup = folium.Popup(iframe, min_width=300, max_width=300)
    folium.Marker(location=list(reversed(ma["geometry"]["coordinates"])), 
                    icon=folium.Icon(color="orange"), 
                    popup=popup).add_to(feature_group_ma)
feature_group_bs.add_to(map)
feature_group_mi.add_to(map)
feature_group_ma.add_to(map)
    
# folium.Choropleth(
#     geo_data = cc_geojson,
#     name="各鄉鎮市區人口",
#     data=population,
#     columns=["TOWNID", "人口密度"],
#     key_on="feature.properties.TOWNID",
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="各鄉鎮市區人口密度",
# ).add_to(map)

folium.LayerControl().add_to(map)
# map

In [ ]:
map

In [88]:
map.save("/Users/kai/Desktop/測試地圖.html")

In [ ]:
ccToolTip = folium.GeoJsonTooltip(["COUNTYNAME", "TOWNNAME", "VILLNAME", "area"], ["縣市", "行政區", "村里", "面積"])

In [ ]:
y = {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}
map = folium.Map([23.973874611, 120.982024611], zoom_start=8)
folium.GeoJson(cc_geojson, style_function=lambda x: y, tooltip=ccToolTip).add_to(map)

In [ ]:
map

In [ ]:
df = pd.read_excel("/Users/kai/Library/CloudStorage/OneDrive-Bridgestone/數據/Google地圖數據/google地圖輪胎店家數據.xlsx")

In [ ]:
geom = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=geom)

In [ ]:
gdf.crs = {'init' :'epsg:4326'}

In [ ]:
villiage = gpd.read_file("/Users/kai/Documents/playground_python/geo_pandas_practice/geo_datasets/mapdata202303230423/TOWN_MOI_1120317.shp", encoding="utf-8")
villiage = villiage.to_crs({"init": "epsg:4326"})

In [ ]:
# 計算polygon的面積需要將geometry轉為projection of Cartesian system，所以需要將geometry使用to_crs({'init': 'epsg:3857'})
villiage_area = villiage.copy()
villiage_area = villiage_area.to_crs({"init": "epsg:3826"})
villiage_area["area"] = round(villiage_area["geometry"].area / 10 ** 6, 2)

In [ ]:
villiage["area"] = villiage_area["area"]

In [ ]:
# villiage_geojson = json.loads(villiage[villiage["COUNTYNAME"] == "臺北市"].to_json())
villiage_geojson = json.loads(villiage.to_json())

In [ ]:
villToolTip = folium.GeoJsonTooltip(["COUNTYNAME", "TOWNNAME", "area"], ["縣市", "行政區", "面積"])

In [ ]:
myMap = folium.Map([22.73444963475145, 120.28458595275877], zoom_start=14)
kaovilliage = folium.GeoJson(villiage_geojson, tooltip=villToolTip).add_to(myMap)
myMap.fit_bounds(kaovilliage.get_bounds())
myMap

In [ ]:
result = gpd.tools.sjoin(gdf, villiage[["COUNTYNAME", "TOWNNAME", "geometry"]], how="left")

In [ ]:
result_df = result.groupby(["COUNTYNAME", "TOWNNAME"]).size().reset_index(name="count")

In [ ]:
result.plot("COUNTYNAME")

In [ ]:
xxx = gdf[gdf["state"] == "台北市"]

In [ ]:
m = folium.Map((25.0133904,121.52245), tiles="Cartodb Positron", zoom_start=11)

In [ ]:
taipei = folium.GeoJson(villiage_geojson, tooltip=villToolTip)

In [ ]:
taipei.add_to(m)

In [ ]:
feature_group = folium.FeatureGroup("markers")

In [ ]:
json.loads(xxx.to_json())["features"][0]["properties"]

In [ ]:
for i in json.loads(xxx.to_json())["features"]:
    if "MICHELIN" in i["properties"]["name"]:
        folium.Marker(location=list(reversed(i["geometry"]["coordinates"])), 
                      icon=folium.Icon(color="blue"), 
                      popup="Michelin").add_to(feature_group)
    elif "普利司通" in i["properties"]["name"]:
        folium.Marker(location=list(reversed(i["geometry"]["coordinates"])), 
                      icon=folium.Icon(color="black"), 
                      popup="Bridgestone").add_to(feature_group)
    else:
        iframe = folium.IFrame("店名:" + str(i["properties"]["name"]) + "<br>" + "地址:" + str(i["properties"]["street"]) + "<br>" + "電話:" + str(i["properties"]["phone"]))
        popup = folium.Popup(iframe, min_width=150, max_width=200)
        folium.Marker(location=list(reversed(i["geometry"]["coordinates"])), 
                      icon=folium.Icon(color="red"), 
                      popup=popup).add_to(feature_group)

In [ ]:
feature_group.add_to(m)

In [ ]:
m.save("test.html")

In [ ]:
# 綜合所得稅
tax = pd.read_csv("https://eip.fia.gov.tw/data/ias/ias109/109_165-9.csv")


In [ ]:
tax["城市"] = tax["鄉鎮市區"].map(lambda x: x[:3])
tax["鄉鎮市區"] = tax["鄉鎮市區"].map(lambda x: x[3:])

In [ ]:
tax_group = tax[(~tax["村里"].isin(["合計", "其他"]))].groupby(["城市", "鄉鎮市區", "村里"]).agg({"綜合所得總額": "sum", "納稅單位(戶)": "sum"}).reset_index()

In [ ]:
tax_group["平均所得"] = (tax_group["綜合所得總額"] /tax_group["納稅單位(戶)"]) * 1000

In [ ]:
tax_group["平均所得"] = tax_group["平均所得"].fillna(0).astype(int)

In [ ]:
tax_group[tax_group["城市"] == "金門縣"].sort_values("平均所得", ascending=False).head(10)

In [ ]:
people.head(3)

In [ ]:
result = pd.merge(tax_group, people, how="left", left_on=["城市", "鄉鎮市區", "村里"], right_on=["COUNTYNAME", "TOWNNAME", "村里名稱"]).drop(["COUNTYNAME", "TOWNNAME", "村里名稱"], axis=1)

In [ ]:
result = result.fillna(0)

In [ ]:
result["納稅比"] = round(result["納稅單位(戶)"] / result["戶數"], 2)

In [ ]:
result.sort_values(by=["平均所得", "綜合所得總額", "納稅比"], ascending=False)

In [ ]:
final = pd.merge(cc, result, how="left", left_on=["COUNTYNAME", "TOWNNAME", "VILLNAME"], right_on=["城市", "鄉鎮市區", "村里"])

In [ ]:
final[["綜合所得總額", "納稅單位(戶)", "平均所得", "戶數", "人口數_合計", "人口數_男", "人口數_女"]] = final[["綜合所得總額", "納稅單位(戶)", "平均所得", "戶數", "人口數_合計", "人口數_男", "人口數_女"]].fillna(0)

In [ ]:
final = final.astype({"綜合所得總額": int, "納稅單位(戶)": int, "平均所得": int, "戶數": int, "人口數_合計": int, "人口數_男": int, "人口數_女": int})

In [ ]:
final["人口密度"] = round(final["人口數_合計"] / final["area"], 0)

In [ ]:
final